In [1]:
#import netCDF4 as nc
import pandas as pd
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import scipy
import plotly.express as px
from scipy.stats import mstats
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
import csv
import fiona
import geopandas
#pd.set_option('display.max_rows', 200)

In [2]:
#loading in mortality dataset from hierarchy file (available on Zenodo)
#fn is the full aggregated dataset
fn = "/projects/kopp/dtb96/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-noadapt.nc4"

#fn1 is the HistClim dataset
fn1 = "/projects/kopp/dtb96/Mortality_Project/data/2_projection/3_impacts/main_specification/raw/single/rcp85/CCSM4/low/SSP3/Agespec_interaction_GMFD_POLY-4_TINV_CYA_NW_w1-oldest-histclim.nc4"

In [3]:
#loading in shapefile associated with hierarchy file
shapefile = "/projects/kopp/dtb96/Mortality_Project/impact-regions-highres-full"

sf = geopandas.read_file(shapefile)
print(sf.head())

    gadmid           hierid  color  ISO geometry
0  28115.0       CAN.1.2.28    1.0  CAN     None
1  28116.0     CAN.1.17.403    2.0  CAN     None
2  28119.0     CAN.2.34.951    3.0  CAN     None
3  28120.0  CAN.11.259.4274    4.0  CAN     None
4  28124.0  CAN.11.269.4448    5.0  CAN     None


In [ ]:
def fix_region(ds):
    """Clean raw mortality Datasets so region is helpful.

    Changes coords/dims/vars so 'region' is region names we can select data by.
    For e.g. `ds.sel(region='IND.10.121')` should work properly.
    """
    return ds.set_coords("regions").swap_dims({"region": "regions"}).rename({"regions": "region"})


ds = fix_region(xr.open_dataset(fn))
ds1 = fix_region(xr.open_dataset(fn1))
ds

In [ ]:

#isolating individual regions to analyze
DEL = ds.sel(region="IND.10.121")   
OSL = ds.sel(region='NOR.12')
CHI = ds.sel(region='USA.14')        #listed as 'Cook' county
SPB = ds.sel(region='BRA')        #double check aggregated version, unaggreagted version not listed as sao paulo
ACC = ds.sel(region='GHA.5')
SYD = ds.sel(region='AUS.4')         #unaggreageted version split into 
BEI = ds.sel(region='CHN.2')        #double check aggregated version, unaggregated version only has Miyun district

#isolating the regions again for the difference calculation down the road
DEL = ds1.sel(region='IND.10.121')   
OSL = ds1.sel(region='NOR.12')
CHI = ds1.sel(region='USA.14')        #listed as 'Cook' county
SPB = ds1.sel(region='BRA')        #double check aggregated version, unaggreagted version not listed as sao paulo
ACC = ds1.sel(region='GHA.5')
SYD = ds1.sel(region='AUS.4')         #unaggreageted version split into 
BEI = ds1.sel(region='CHN.2')

In [ ]:
q = (0.17, 0.5, 0.83)
window_n = 30
rebased_qs = ds["rebased"].rolling({"year": window_n}, center=False).construct("tmp").quantile(q,skipna= False, dim="tmp")
rebased_qs1 = ds1["rebased"].rolling({"year": window_n}, center=False).construct("tmp").quantile(q,skipna= False, dim="tmp")

In [ ]:
##can use rebased_qs.sel(quantile=0.5, region="IND.10.121") to grab something
rebased_qs.stack(z=["region", "quantile"]).to_pandas()
#rebased_qs.sel(region="IND.10.121", quantile =0.17)

In [ ]:
rebased_qs.sel(region="BWA.7").plot.line(x="year")

In [ ]:
##original code from Brewster
##threshold = rebased_qs.sel(quantile=0.17, year=window).max(dim="year")
##mask_years_over_threshold = rebased_qs.sel(quantile=0.83, year=window) <= threshold
##earliest_threshold_year = mask_years_over_threshold.idxmax(dim="year")

In [ ]:
window = slice(2010, 2100)

threshold = rebased_qs.sel(quantile=0.17, year=2010)
mask_years_over_threshold = rebased_qs.sel(quantile=0.83, year=window) <= threshold
earliest_threshold_year = mask_years_over_threshold.idxmax(dim="year")
array1 = earliest_threshold_year['region'].values
array2 = earliest_threshold_year.values


---
# 230922 pk

In [ ]:
def fun_threshold(quant,yr):
    threshold = rebased_qs.sel(quantile=quant, year=yr)
    return threshold

In [ ]:
mask_years_over_threshold1 = fun_threshold(0.83, window) <= fun_threshold(0.17, 2100)
earliest_threshold_year1 = mask_years_over_threshold1.idxmax(dim="year")

In [ ]:
mask_years_over_threshold2 = fun_threshold(0.17, window) <= fun_threshold(0.83, 2100)
earliest_threshold_year2 = mask_years_over_threshold2.idxmax(dim="year")

---
# ^^^pk

In [ ]:
if:
    threshold(q=0.17,year=2000) -- first qunatile value
    mask_years_over_threshold = rebased_qs.sel(quantile=0.83, year=window) <= threshold -
    ##says if first 0.17 percntile becomes greater than 0.83 percentile at some year Y, damages occur (mortality increasin)
    ##opposite true of first 0.83 crosses 0.17 - benefits occur
    ##in some regions neither benefits nor damages emerge 
    #threshold 1 will find what year the 2010 0.17 percentile crosses 0.83
    #threshold 2 will find what year 2010 0.83 percnetile cross 0.17
    

In [ ]:
array1.shape

In [ ]:
threshold2 = rebased_qs.sel(quantile=0.83, year=2010)
mask_years_over_threshold2 = rebased_qs.sel(quantile=0.17, year=window) <= threshold2
earliest_threshold_year2 = mask_years_over_threshold2.idxmax(dim="year")

In [ ]:
rebased_qs.sel(quantile[0=0.17, year=2010)

In [ ]:
sf['TOE_benefits'] = earliest_threshold_year
sf['TOE_damages'] = earliest_threshold_year2

In [ ]:
# Plot the GeoDataFrame
fig, ax = plt.subplots(figsize=(10, 10))
sf.plot(column="TOE_damages",ax=ax,legend=True)

In [ ]:
# Plot the GeoDataFrame
fig, ax = plt.subplots(figsize=(10, 10))
sf.plot(column="TOE_benefits",ax=ax,legend=True)